In [8]:
import os, sys, shutil, random
HOME = os.getcwd()
import numpy as np
from IPython.display import display, Image
from IPython import display
import yaml
import glob
import cv2
import utils
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
display.clear_output()
display.clear_output()
!yolo checks

Ultralytics YOLOv8.2.42 🚀 Python-3.8.10 torch-2.2.1+cu121 CUDA:0 (Quadro RTX 8000, 48585MiB)
Setup complete ✅ (72 CPUs, 250.5 GB RAM, 373.6/1876.2 GB disk)

OS                  Linux-5.15.0-113-generic-x86_64-with-glibc2.29
Environment         Linux
Python              3.8.10
Install             pip
RAM                 250.54 GB
CPU                 Intel Xeon Gold 5220 2.20GHz
CUDA                12.1

numpy               ✅ 1.24.4<2.0.0,>=1.23.5
matplotlib          ✅ 3.7.5>=3.3.0
opencv-python       ✅ 4.8.0.74>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.10.1>=1.4.1
torch               ✅ 2.2.1>=1.8.0
torchvision         ✅ 0.17.1>=0.9.0
tqdm                ✅ 4.66.2>=4.64.0
psutil              ✅ 5.9.8
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.0.3>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0
ultralytics-thop    ✅ 2.0.0>=2.0.0


In [ ]:
HOME

In [6]:
input_directory = "train"
output_directory = HOME + "/input"
utils.split_data(input_directory, output_directory, categories=['cell', 'nuclei'])

train_images_dir: train/images
train_masks_dir: train/masks
Processing category: cell
Category cell - Train: 637, Val: 159
Processing category: nuclei
Mask file does not exist: train/masks/nuclei/547_img.png
Mask file does not exist: train/masks/nuclei/548_img.png
Mask file does not exist: train/masks/nuclei/549_img.png
Mask file does not exist: train/masks/nuclei/550_img.png
Mask file does not exist: train/masks/nuclei/551_img.png
Mask file does not exist: train/masks/nuclei/552_img.png
Mask file does not exist: train/masks/nuclei/553_img.png
Mask file does not exist: train/masks/nuclei/554_img.png
Mask file does not exist: train/masks/nuclei/555_img.png
Mask file does not exist: train/masks/nuclei/556_img.png
Mask file does not exist: train/masks/nuclei/557_img.png
Mask file does not exist: train/masks/nuclei/558_img.png
Mask file does not exist: train/masks/nuclei/560_img.png
Mask file does not exist: train/masks/nuclei/565_img.png
Mask file does not exist: train/masks/nuclei/567_im

In [21]:
HOME
train_masks_path= HOME+  "/input/train_masks"
train_output_dir =HOME + "/input/train_images"
utils.process_masks(mask_paths=train_masks_path,
              output_dir=train_output_dir,
              category_ids={'cell':1, 'nuclei':2}, 
              MASKS_EXT='png',
              ORIGINAL_EXT='png')

Created 94853 annotations for images in folder: /home/kamenan/Documents/Stage Dev Deep Learning/YOLOV8 evaluation/Multi-label experiment/input/train_masks


In [20]:
val_masks_path= HOME+  "/input/val_masks"
val_output_dir =HOME + "/input/val_images"
utils.process_masks(mask_paths=val_masks_path,
              output_dir=val_output_dir,
              category_ids={'cell':1, 'nuclei':2}, 
              MASKS_EXT='png',
              ORIGINAL_EXT='png')

Created 23045 annotations for images in folder: /home/kamenan/Documents/Stage Dev Deep Learning/YOLOV8 evaluation/Multi-label experiment/input/val_masks


In [17]:
base_input_path = "input/"
base_output_path = "yolo_dataset/"

In [18]:
# Processing training dataset 
utils.convert_to_yolo(
    input_images_path=os.path.join(base_input_path, "train_images"),
    input_json_path=os.path.join(base_input_path, "train_images/coco_annotations.json"),
    output_images_path=os.path.join(base_output_path, "train/images"),
    output_labels_path=os.path.join(base_output_path, "train/labels")
)

In [23]:
# Processing validation dataset (if needed)
utils.convert_to_yolo(
    input_images_path=os.path.join(base_input_path, "val_images"),
    input_json_path=os.path.join(base_input_path, "val_images/coco_annotations.json"),
    output_images_path=os.path.join(base_output_path, "valid/images"),
    output_labels_path=os.path.join(base_output_path, "valid/labels")
)

In [24]:
# Creating the YAML configuration file
create_yaml(
    input_json_path=os.path.join(base_input_path, "train_images/coco_annotations.json"),
    output_yaml_path=os.path.join(base_output_path, "data.yaml"),
    train_path="train/images",
    val_path="valid/images",
    test_path='../test/images'  # or None if not applicable
)

In [25]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [9]:
#display_images_with_annotations(image_paths, annotation_paths)

In [2]:
model = YOLO("yolov8m-seg.yaml") # build a new model from yaml
model = YOLO("yolov8m-seg.pt") # transfer weights fro a pre-trained model (recommended for training)

In [5]:
with open('yolo_dataset/data.yaml', 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [6]:
multi_class = "yolo_dataset/results"
name = "125_epochs_seg_m"

In [10]:
# Train the model
#results = model.train(data='yolo_dataset/data.yaml',
#                    project=multi_class,
#                    name=name,
#                    epochs=125
#                    )
     

In [ ]:
def evaluation(model, data_dir, ious):

    #ious = np.linspace(.5,1,50)
    precision = []
    recall = []
    jac_idx = []
    for io in ious:
        tp_eval = YOLO(model)
        metrics = tp_eval.val(data= data_dir,
                              iou=io,
                              device=0
                              )
        precision.append(metrics.results_dict["metrics/precision(B)"])
        recall.append(metrics.results_dict["metrics/recall(B)"])
        conf_mtrx = metrics.confusion_matrix
        j_idx = conf_mtrx.matrix[0][0]/conf_mtrx.matrix.sum()
        jac_idx.append(j_idx)
    return jac_idx, precision, recall

In [ ]:
jac_idx, precision, recall = evaluation(model="yolo_dataset/results/125_epochs_seg_l2/weights/best.pt", data_dir="yolo_dataset/data.yaml", ious=np.linspace(.5,1,50))


In [ ]:
model1 = YOLO("yolo_dataset/results/125_epochs_seg_l2/weights/best.pt")

image = "/home/kamenan/Documents/Stage Dev Deep Learning/YOLOV8 evaluation/sanofi_test/test_img/200L_GBD_50K_029.dm3.tif.tif"

results = model1.predict(image, conf=0.5, show_labels=True, show_boxes=True, show=True)  #Adjust conf threshold